In [1]:
import sys
print(sys.executable)

f:\MachinL\.venv\Scripts\python.exe


In [2]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


//1.Tiền xử lý dữ liệu

In [16]:
df = pd.read_csv('epl_24-25.csv')
df.head()

,Season,MatchDate,HomeTeam,AwayTeam,FullTimeHomeGoals,FullTimeAwayGoals,FullTimeResult,HalfTimeHomeGoals,HalfTimeAwayGoals,HalfTimeResult,...,HomeShotsOnTarget,AwayShotsOnTarget,HomeCorners,AwayCorners,HomeFouls,AwayFouls,HomeYellowCards,AwayYellowCards,HomeRedCards,AwayRedCards
0,2023/24,8/11/2023,Burnley,Man City,0,3,A,0,2,A,...,1,8,6,5,11,8,0,0,1,0
1,2023/24,8/12/2023,Arsenal,Nott'm Forest,2,1,H,2,0,H,...,7,2,8,3,12,12,2,2,0,0
2,2023/24,8/12/2023,Bournemouth,West Ham,1,1,D,0,0,D,...,5,3,10,4,9,14,1,4,0,0
3,2023/24,8/12/2023,Brighton,Luton,4,1,H,1,0,H,...,12,3,6,7,11,12,2,2,0,0
4,2023/24,8/12/2023,Everton,Fulham,0,1,A,0,0,D,...,9,2,10,4,12,6,0,2,0,0


In [17]:
def select_feature(dff, feature_names):
    return dff[feature_names].values

def select_label(dff, label_name):
    return dff[label_name].values

//a. encode doi bong

In [18]:
teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()
team_to_id = {team: idx for idx, team in enumerate(sorted(teams))}

df['HomeTeamID'] = df['HomeTeam'].map(team_to_id)
df['AwayTeamID'] = df['AwayTeam'].map(team_to_id)


//b. tinh tu bang

In [ ]:
df['HomeShotAccuracy'] = df['HomeShotsOnTarget'] / (df['HomeShots'] + 1)
df['AwayShotAccuracy'] = df['AwayShotsOnTarget'] / (df['AwayShots'] + 1)

df['ShotDiff'] = df['HomeShots'] - df['AwayShots']

df['DisciplineHome'] = df['HomeYellowCards'] + 3 * df['HomeRedCards']
df['DisciplineAway'] = df['AwayYellowCards'] + 3 * df['AwayRedCards']


//c.Feature lich su

In [80]:
import pandas as pd

def compute_team_last5_stats(df, team_col, is_home):
    df_sorted = df.sort_values("MatchDate")
    teams = df[team_col].unique()
    history = {team: [] for team in teams}

    # Các list lưu feature
    win_rate_5 = []
    draw_rate_5 = []
    avg_goals_5 = []
    avg_conceded_5 = []
    homeaway_win_rate_5 = []
    avg_sot_5 = []
    avg_shots_conceded_5 = []

    for idx, row in df_sorted.iterrows():
        team = row[team_col]
        recent = history[team][-5:]

        if len(recent) < 5:
            win_rate_5.append(0)
            draw_rate_5.append(0)
            avg_goals_5.append(0)
            avg_conceded_5.append(0)
            homeaway_win_rate_5.append(0)
            avg_sot_5.append(0)
            avg_shots_conceded_5.append(0)
        else:
            df_last5 = pd.DataFrame(recent)
            total = len(df_last5)
            win_rate_5.append((df_last5['result'] == 1).sum() / total)
            draw_rate_5.append((df_last5['result'] == 0).sum() / total)
            avg_goals_5.append(df_last5['goals'].mean())
            avg_conceded_5.append(df_last5['conceded'].mean())
            homeaway_win_rate_5.append(df_last5['homeaway_win'].mean())
            avg_sot_5.append(df_last5['sot'].mean())
            avg_shots_conceded_5.append(df_last5['shots_conceded'].mean())

        # Trận hiện tại
        if is_home:
            goals = row['FullTimeHomeGoals']
            conceded = row['FullTimeAwayGoals']
            sot = row['HomeShotsOnTarget']
            shots_conceded = row['AwayShots']
        else:
            goals = row['FullTimeAwayGoals']
            conceded = row['FullTimeHomeGoals']
            sot = row['AwayShotsOnTarget']
            shots_conceded = row['HomeShots']

        # Kết quả H-D-A: 1=HomeWin, 0=Draw, 2=AwayWin
        if goals > conceded:
            result = 1
        elif goals == conceded:
            result = 0
        else:
            result = 2

        homeaway_win = 1 if (is_home and result == 1) or (not is_home and result == 1) else 0

        history[team].append({
            'result': result,
            'goals': goals,
            'conceded': conceded,
            'homeaway_win': homeaway_win,
            'sot': sot,
            'shots_conceded': shots_conceded
        })

    return (win_rate_5, draw_rate_5, avg_goals_5, avg_conceded_5,
            homeaway_win_rate_5, avg_sot_5, avg_shots_conceded_5)


def compute_head2head_counts(df):
    df_sorted = df.sort_values("MatchDate")
    history = {}  # key = (HomeTeam, AwayTeam)
    h2h_home_win = []
    h2h_draw = []
    h2h_away_win = []

    for idx, row in df_sorted.iterrows():
        teams_pair = (row['HomeTeam'], row['AwayTeam'])
        recent = history.get(teams_pair, [])

        # Số trận thắng-hòa-thua trước đó
        home_w = sum(1 for r in recent if r == 1)
        draw = sum(1 for r in recent if r == 0)
        away_w = sum(1 for r in recent if r == 2)

        h2h_home_win.append(home_w)
        h2h_draw.append(draw)
        h2h_away_win.append(away_w)

        # Kết quả trận hiện tại
        if row['FullTimeHomeGoals'] > row['FullTimeAwayGoals']:
            result = 1
        elif row['FullTimeHomeGoals'] == row['FullTimeAwayGoals']:
            result = 0
        else:
            result = 2

        if teams_pair not in history:
            history[teams_pair] = []
        history[teams_pair].append(result)

    return h2h_home_win, h2h_draw, h2h_away_win


# --- Tính feature ---
# Đội chủ nhà
(df['HomeWinRate5'], df['HomeDrawRate5'], df['HomeAvgGoals5'],
 df['HomeAvgGoalsConceded5'], df['HomeHomeWinRate5'],
 df['HomeAvgShotOnTarget5'], df['HomeAvgShotsConceded5']) = compute_team_last5_stats(df, 'HomeTeam', True)

# Đội khách
(df['AwayWinRate5'], df['AwayDrawRate5'], df['AwayAvgGoals5'],
 df['AwayAvgGoalsConceded5'], df['AwayAwayWinRate5'],
 df['AwayAvgShotOnTarget5'], df['AwayAvgShotsConceded5']) = compute_team_last5_stats(df, 'AwayTeam', False)

# Lịch sử đối đầu 5 trận gần nhất
(df['Head2Head_HomeWin'], df['Head2Head_Draw'], df['Head2Head_AwayWin']) = compute_head2head_counts(df)

# Labels H-D-A
def encode_result(row):
    if row['FullTimeHomeGoals'] > row['FullTimeAwayGoals']:
        return 1
    elif row['FullTimeHomeGoals'] == row['FullTimeAwayGoals']:
        return 0
    else:
        return 2

df['Result'] = df.apply(encode_result, axis=1)


In [172]:
team_tier = {
    'Arsenal': 5,
    'Man City': 5,
    'Liverpool': 5,
    'Chelsea': 5,
    'Man United': 5,
    'Tottenham': 4,
    'Aston Villa': 4,
    'West Ham': 4,
    'Wolves': 4,
    'Newcastle': 4,
    'Brighton': 3,
    'Crystal Palace': 3,
    'Leicester': 3,
    'Fulham': 3,
    'Everton': 3,
    'Bournemouth': 2,
    'Burnley': 2,
    'Ipswich': 2,
    'Southampton': 2,
    'Nott\'m Forest': 2,
    'Sheffield United': 1
}

# Gán default tier = 2 cho những đội còn lại
df['HomeTeamTier'] = df['HomeTeam'].map(lambda x: team_tier.get(x, 2))
df['AwayTeamTier'] = df['AwayTeam'].map(lambda x: team_tier.get(x, 2))

In [194]:
def compute_elo(df, K=40, home_advantage=50):
    df = df.sort_values("MatchDate").reset_index(drop=True)

    # Khởi tạo Elo tất cả đội = 1500
    teams = pd.unique(df[['HomeTeam','AwayTeam']].values.ravel())
    elo = {team:1500 for team in teams}

    home_elos = []
    away_elos = []
    home_expected = []
    away_expected = []

    for idx, row in df.iterrows():
        home = row["HomeTeam"]
        away = row["AwayTeam"]

        R_home = elo[home]
        R_away = elo[away]

        # Expected score
        diff = (R_home + home_advantage) - R_away
        expected_home = 1 / (1 + 10 ** (-diff / 400))
        expected_away = 1 - expected_home

        home_expected.append(expected_home)
        away_expected.append(expected_away)

        home_elos.append(R_home)
        away_elos.append(R_away)

        # Actual result
        if row["FullTimeHomeGoals"] > row["FullTimeAwayGoals"]:
            score_home = 1
        elif row["FullTimeHomeGoals"] < row["FullTimeAwayGoals"]:
            score_home = 0
        else:
            score_home = 0.5

        score_away = 1 - score_home

        # Update Elo
        elo[home] = R_home + K * (score_home - expected_home)
        elo[away] = R_away + K * (score_away - expected_away)

    df["HomeElo"] = home_elos
    df["AwayElo"] = away_elos
    df["HomeEloExpected"] = home_expected
    df["AwayEloExpected"] = away_expected

    return df


In [195]:
df = compute_elo(df)


In [196]:
df.iloc[300:310] 

,Season,MatchDate,HomeTeam,AwayTeam,FullTimeHomeGoals,FullTimeAwayGoals,FullTimeResult,HalfTimeHomeGoals,HalfTimeAwayGoals,HalfTimeResult,...,Head2Head_AwayWinRate5,Head2Head_HomeWin,Head2Head_Draw,Head2Head_AwayWin,HomeTeamTier,AwayTeamTier,HomeElo,AwayElo,HomeEloExpected,AwayEloExpected
300,2023/24,12/6/2023,Aston Villa,Man City,1,0,H,0,0,D,...,0,0,0,0,4,5,1519.349376,1580.346158,0.484180,0.515820
301,2023/24,12/7/2023,Everton,Newcastle,3,0,H,0,0,D,...,0,0,0,0,3,4,1493.692437,1577.821635,0.451042,0.548958
302,2023/24,12/7/2023,Tottenham,West Ham,1,2,A,1,0,H,...,0,0,0,0,4,4,1498.993898,1504.357547,0.563886,0.436114
303,2024/25,12/7/2024,Aston Villa,Southampton,1,0,H,1,0,H,...,0,0,0,1,4,2,1539.982190,1346.163861,0.802744,0.197256
304,2024/25,12/7/2024,Brentford,Newcastle,4,2,H,2,2,D,...,0,0,0,1,2,4,1419.969667,1555.863296,0.378847,0.621153
305,2024/25,12/7/2024,Crystal Palace,Man City,2,2,D,1,1,D,...,0,0,0,0,3,5,1488.011259,1559.713344,0.468809,0.531191
306,2024/25,12/7/2024,Man United,Nott'm Forest,2,3,A,1,1,D,...,0,0,0,0,5,2,1516.958195,1521.935927,0.564432,0.435568
307,2024/25,12/8/2024,Fulham,Arsenal,1,1,D,1,0,H,...,0,0,0,0,3,5,1523.021849,1623.276288,0.428178,0.571822
308,2024/25,12/8/2024,Leicester,Brighton,2,2,D,0,1,A,...,0,1,0,0,3,3,1404.732669,1507.512321,0.424623,0.575377
309,2024/25,12/8/2024,Ipswich,Bournemouth,1,2,A,1,0,H,...,0,0,1,0,2,2,1426.098196,1571.702334,0.365785,0.634215


//d. encode ket qua tran dau

In [174]:
def encode_result(row):
    if row['FullTimeHomeGoals'] > row['FullTimeAwayGoals']:
        return 1  # home win
    elif row['FullTimeHomeGoals'] < row['FullTimeAwayGoals']:
        return 2  # away win
    return 0  # draw

df['Result'] = df.apply(encode_result, axis=1)

In [87]:

df.head()

,Season,MatchDate,HomeTeam,AwayTeam,FullTimeHomeGoals,FullTimeAwayGoals,FullTimeResult,HalfTimeHomeGoals,HalfTimeAwayGoals,HalfTimeResult,...,AwayAwayWinRate5,Result,HomeDrawRate5,AwayDrawRate5,Head2Head_HomeWinRate5,Head2Head_DrawRate5,Head2Head_AwayWinRate5,Head2Head_HomeWin,Head2Head_Draw,Head2Head_AwayWin
0,2023/24,8/11/2023,Burnley,Man City,0,3,A,0,2,A,...,0.0,2,0.0,0.0,0,0,0,0,0,0
1,2023/24,8/12/2023,Arsenal,Nott'm Forest,2,1,H,2,0,H,...,0.0,1,0.0,0.0,0,0,0,0,0,0
2,2023/24,8/12/2023,Bournemouth,West Ham,1,1,D,0,0,D,...,0.0,0,0.0,0.0,0,0,0,0,0,0
3,2023/24,8/12/2023,Brighton,Luton,4,1,H,1,0,H,...,0.0,1,0.0,0.0,0,0,0,0,0,0
4,2023/24,8/12/2023,Everton,Fulham,0,1,A,0,0,D,...,0.0,2,0.0,0.0,0,0,0,0,0,0


//e. Lay Feature va labels

In [229]:
feature_columns = [
    # Lịch sử 5 trận gần nhất
    "HomeWinRate5", "AwayWinRate5",
    "HomeDrawRate5", "AwayDrawRate5",
    "HomeAvgGoals5", "AwayAvgGoals5",
    "HomeAvgGoalsConceded5", "AwayAvgGoalsConceded5",
    "HomeHomeWinRate5", "AwayAwayWinRate5",
    "HomeAvgShotOnTarget5", "AwayAvgShotOnTarget5",
    "HomeAvgShotsConceded5", "AwayAvgShotsConceded5",
    "HomeTeamTier",	"AwayTeamTier",
        # Elo mới thêm
    "HomeElo",
    "AwayElo",
    "HomeEloExpected",
    "AwayEloExpected",
    

]

# Tách feature và label
features = select_feature(df, feature_columns)
label = select_label(df, "Result")  # cột Result là 1-0-2 encode thắng hòa thua

2.Train model

//a.Chia du lieu

In [230]:


# 1. Chia train+val vs test (vd 80% train+val, 20% test)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    features, label, test_size=0.2, random_state=42, stratify=label
)

# 2. Chia train vs val (vd 75% train, 25% val của phần trainval)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, random_state=42, stratify=y_trainval
)

//b.XGBost

In [231]:
from xgboost.callback import EarlyStopping


In [234]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'multi:softmax',
    'num_class': 3,
    'max_depth': 3,
    'eta': 0.02,
    'eval_metric': 'mlogloss',
    'seed': 42
}

watchlist = [(dtrain, 'train'), (dval, 'eval')]

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=300,
    evals=watchlist,
    early_stopping_rounds=10
)


[0]	train-mlogloss:1.07602	eval-mlogloss:1.07790
[1]	train-mlogloss:1.07067	eval-mlogloss:1.07501
[2]	train-mlogloss:1.06555	eval-mlogloss:1.07180
[3]	train-mlogloss:1.06044	eval-mlogloss:1.06907
[4]	train-mlogloss:1.05570	eval-mlogloss:1.06589
[5]	train-mlogloss:1.05086	eval-mlogloss:1.06340
[6]	train-mlogloss:1.04623	eval-mlogloss:1.06057
[7]	train-mlogloss:1.04169	eval-mlogloss:1.05808
[8]	train-mlogloss:1.03737	eval-mlogloss:1.05588
[9]	train-mlogloss:1.03310	eval-mlogloss:1.05329
[10]	train-mlogloss:1.02889	eval-mlogloss:1.05101
[11]	train-mlogloss:1.02481	eval-mlogloss:1.04863
[12]	train-mlogloss:1.02067	eval-mlogloss:1.04665
[13]	train-mlogloss:1.01692	eval-mlogloss:1.04440
[14]	train-mlogloss:1.01275	eval-mlogloss:1.04291
[15]	train-mlogloss:1.00888	eval-mlogloss:1.04087
[16]	train-mlogloss:1.00489	eval-mlogloss:1.03938
[17]	train-mlogloss:1.00099	eval-mlogloss:1.03762
[18]	train-mlogloss:0.99738	eval-mlogloss:1.03585
[19]	train-mlogloss:0.99366	eval-mlogloss:1.03437
[20]	train

In [235]:
# Chuyển X_test sang DMatrix
dtest = xgb.DMatrix(X_test)

# Dự đoán trên test set
y_pred = bst.predict(dtest)

In [237]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.5068493150684932


In [238]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=['Draw','HomeWin','AwayWin']))

              precision    recall  f1-score   support

        Draw       0.00      0.00      0.00        34
     HomeWin       0.51      0.83      0.63        64
     AwayWin       0.54      0.44      0.48        48

    accuracy                           0.51       146
   macro avg       0.35      0.42      0.37       146
weighted avg       0.40      0.51      0.44       146

